In [26]:
%pip install --upgrade langchain langchain-google-genai google-generativeai


  Using cached langchain_google_genai-2.1.10-py3-none-any.whl.metadata (7.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pr


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\crish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [45]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

API_IA = os.getenv("GOOGLE_API_KEY")

Conexão com o Gemini

In [46]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0,
    api_key = API_IA
)

In [47]:
resp_text = llm.invoke("o que é agentes em IA ?")
print(resp_text.content)

Em Inteligência Artificial (IA), um **agente** é uma entidade autônoma que percebe seu ambiente através de sensores e age sobre esse ambiente através de atuadores, com o objetivo de alcançar suas metas ou maximizar uma medida de desempenho.

Pense nele como um "ser" (seja ele um software ou um robô físico) que interage com o mundo ao seu redor de forma inteligente.

### Componentes Chave de um Agente em IA:

1.  **Ambiente (Environment):** É o mundo no qual o agente opera. Pode ser um ambiente físico (como uma sala para um robô aspirador) ou um ambiente virtual (como um jogo para um agente de IA, ou a internet para um bot de busca).

2.  **Sensores (Sensors):** São os meios pelos quais o agente percebe o estado do ambiente.
    *   **Exemplos físicos:** Câmeras, microfones, sensores de toque, GPS.
    *   **Exemplos virtuais:** Entradas de teclado, dados de um banco de dados, pixels de uma tela, APIs.

3.  **Atuadores (Actuators):** São os meios pelos quais o agente age sobre o ambient

In [48]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."

    """
    decisao: AUTO_RESOLVER
    urgencia: BAIXA
    campo_faltante: []
    """
)

In [49]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [50]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0,
    api_key = API_IA
)

In [51]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])
    return saida.model_dump()



In [ ]:
testes =["Posso reembolsar a internet",
         "Quero mais 5 dias de trabalho remoto como faço?",
         "já exitio dragoes"]

In [53]:
for msg_text in testes:
    print(f"Pergunta: {msg_text}\n -> Resposta: {triagem(msg_text)}\n")

Pergunta: Posso reembolsar a internet
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remo como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: já exitio dragoes
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



In [36]:
%pip install --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\crish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [54]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("C:\DEV\Agentes de IA Google\documentos").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado arquivo com sucesso {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")


Carregado arquivo com sucesso Política de Reembolsos (Viagens e Despesas).pdf
Carregado arquivo com sucesso Política de Uso de E-mail e Segurança da Informação.pdf
Carregado arquivo com sucesso Políticas de Home Office.pdf
Total de documentos carregados: 3


In [55]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

spliter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = spliter.split_documents(docs)



In [56]:
for chunk in chunks:
    print(chunk)
    print("-----------------------------------------------")

page_content='Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'C:\\DEV\\Agentes de IA Google\\documentos\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'C:\\DEV\\Agentes de IA Google\\documentos\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
-----------------------------------------------
page_content='são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Interne

In [57]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key= API_IA
)

In [59]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(chunks, embedings)
retriver = vectorstore.as_retriever(search_type= "similarity_score_threshold", search_kwargs={"score_threshold": 0.3, "k": 4})



In [62]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}") 
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)


In [63]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    doccs_relaconados = retriver.invoke(pergunta)

    if not doccs_relaconados:
        return {"answer": "Não sei",
                "citacoes": [],
                "contexto_encontrado": False}
    answer = document_chain.invoke({"input": pergunta,
                                    "context": doccs_relaconados})
    
    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei",
                "citacoes": [],
                "contexto_encontrado": False}
    
    return {"answer": txt,
            "citacoes": doccs_relaconados,
            "contexto_encontrado": True}

In [64]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [65]:
for msg_text in testes:
    resposta = perguntar_politica_RAG(msg_text)
    print(f"Pergunta: {msg_text}")
    print(f"Resposta: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print(f"Citações:")
        print(f"{resposta['citacoes']}")
        print("---------------------------------")

Pergunta: Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Citações:
[Document(id='f2a6893f-bda2-47b1-8974-ef371366c0a0', metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'C:\\DEV\\Agentes de IA Google\\documentos\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'C:\\DEV\\Agentes de IA Google\\documentos\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='são reembolsáveis.\u200b\n \n3.\u200b Transporte: táxi/app são permitidos quando não houver alternativa viável. \nComprovantes obrigatórios.\u200b\n \n4.\u200b Internet para home office: reembolsável via subsídio mensal de até R$